In [1]:
import sys
sys.version

'3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]'

In [68]:
import tetravolume as tv
import flextegrity as fx
import pandas as pd
import numpy as np
from importlib import reload
import sympy

In [3]:
fx.Tetrahedron

flextegrity.Tetrahedron

In [4]:
import qrays
import flextegrity

In [67]:
reload(flextegrity)

<module 'flextegrity' from '/Users/kirbyurner/Documents/School_of_Tomorrow/flextegrity.py'>

In [6]:
fx.Tetrahedron().vertexes

{'a': ivm_vector(a=1, b=0, c=0, d=0),
 'b': ivm_vector(a=0, b=1, c=0, d=0),
 'c': ivm_vector(a=0, b=0, c=1, d=0),
 'd': ivm_vector(a=0, b=0, c=0, d=1)}

In [7]:
type(fx.Tetrahedron().vertexes['a'].b)

sympy.core.numbers.Zero

In [8]:
type(fx.A.b)

sympy.core.numbers.Zero

In [9]:
arrays = [
    coordsys := ["IVM", "IVM", "IVM", "IVM", "XYZ", "XYZ", "XYZ"],
    coordnms := ["A", "B", "C", "D", "X", "Y", "Z"]]
tuples = list(zip(*arrays))
tuples

[('IVM', 'A'),
 ('IVM', 'B'),
 ('IVM', 'C'),
 ('IVM', 'D'),
 ('XYZ', 'X'),
 ('XYZ', 'Y'),
 ('XYZ', 'Z')]

In [10]:
index = pd.MultiIndex.from_tuples(tuples, names=["System", "Coords"])

In [11]:
vs = fx.Cuboctahedron().vertexes.items()
rows = []
for label, v in vs:
    xyz = v.xyz
    rows.append(dict(name = label, A=v.a, B=v.b, C=v.c, D = v.d, X = xyz.x, Y = xyz.y, Z = xyz.z))

In [12]:
df = pd.DataFrame.from_dict(rows)
df.index = df.name
df.drop(columns="name", axis=1, inplace=True)
df.columns = index
df

System IVM                  XYZ                        
Coords   A  B  C  D           X           Y           Z
name                                                   
o        2  1  1  0           0   sqrt(2)/2   sqrt(2)/2
p        2  1  0  1   sqrt(2)/2           0   sqrt(2)/2
q        1  2  1  0  -sqrt(2)/2           0   sqrt(2)/2
r        1  2  0  1           0  -sqrt(2)/2   sqrt(2)/2
s        1  0  2  1           0   sqrt(2)/2  -sqrt(2)/2
t        1  0  1  2   sqrt(2)/2           0  -sqrt(2)/2
u        0  1  2  1  -sqrt(2)/2           0  -sqrt(2)/2
v        0  1  1  2           0  -sqrt(2)/2  -sqrt(2)/2
w        1  1  2  0  -sqrt(2)/2   sqrt(2)/2           0
x        0  2  1  1  -sqrt(2)/2  -sqrt(2)/2           0
y        1  1  0  2   sqrt(2)/2  -sqrt(2)/2           0
z        2  0  1  1   sqrt(2)/2   sqrt(2)/2           0

In [13]:
type(df.loc['o'].IVM.A)

sympy.core.numbers.Integer

In [14]:
df

System IVM                  XYZ                        
Coords   A  B  C  D           X           Y           Z
name                                                   
o        2  1  1  0           0   sqrt(2)/2   sqrt(2)/2
p        2  1  0  1   sqrt(2)/2           0   sqrt(2)/2
q        1  2  1  0  -sqrt(2)/2           0   sqrt(2)/2
r        1  2  0  1           0  -sqrt(2)/2   sqrt(2)/2
s        1  0  2  1           0   sqrt(2)/2  -sqrt(2)/2
t        1  0  1  2   sqrt(2)/2           0  -sqrt(2)/2
u        0  1  2  1  -sqrt(2)/2           0  -sqrt(2)/2
v        0  1  1  2           0  -sqrt(2)/2  -sqrt(2)/2
w        1  1  2  0  -sqrt(2)/2   sqrt(2)/2           0
x        0  2  1  1  -sqrt(2)/2  -sqrt(2)/2           0
y        1  1  0  2   sqrt(2)/2  -sqrt(2)/2           0
z        2  0  1  1   sqrt(2)/2   sqrt(2)/2           0

In [15]:
def tabulate(shape, dec=False, n=8):
    vs = shape.vertexes.items()
    rows = []
    for label, v in vs:
        xyz = v.xyz
        if not dec:
            rows.append(dict(name = label, 
                             A=v.a, 
                             B=v.b, 
                             C=v.c, 
                             D = v.d, 
                             X = xyz.x, 
                             Y = xyz.y, 
                             Z = xyz.z))
        else:
            rows.append(dict(name = label, 
                             A = v.a.evalf(n).round(n), 
                             B = v.b.evalf(n).round(n), 
                             C = v.c.evalf(n).round(n), 
                             D = v.d.evalf(n).round(n), 
                             X = xyz.x.evalf(n).round(n), 
                             Y = xyz.y.evalf(n).round(n), 
                             Z = xyz.z.evalf(n).round(n)))            
    df = pd.DataFrame.from_dict(rows)
    df.index = df.name
    df.drop(columns="name", axis=1, inplace=True)
    df.columns = index
    return df

In [16]:
tabulate(fx.Tetrahedron())

System IVM                  XYZ                        
Coords   A  B  C  D           X           Y           Z
name                                                   
a        1  0  0  0   sqrt(2)/4   sqrt(2)/4   sqrt(2)/4
b        0  1  0  0  -sqrt(2)/4  -sqrt(2)/4   sqrt(2)/4
c        0  0  1  0  -sqrt(2)/4   sqrt(2)/4  -sqrt(2)/4
d        0  0  0  1   sqrt(2)/4  -sqrt(2)/4  -sqrt(2)/4

In [17]:
df = tabulate(fx.Icosahedron(),dec=True,n=5)
df

System      IVM                                  XYZ                    
Coords        A        B        C        D         X         Y         Z
name                                                                    
o        1.8512  0.70711   1.1441        0       0.0   0.80902   0.50000
p        1.8512   1.1441        0  0.70711   0.50000       0.0   0.80902
q        1.1441   1.8512  0.70711        0  -0.50000       0.0   0.80902
r       0.70711   1.8512        0   1.1441       0.0  -0.80902   0.50000
s        1.1441        0   1.8512  0.70711       0.0   0.80902  -0.50000
t       0.70711        0   1.1441   1.8512   0.50000       0.0  -0.80902
u             0  0.70711   1.8512   1.1441  -0.50000       0.0  -0.80902
v             0   1.1441  0.70711   1.8512       0.0  -0.80902  -0.50000
w       0.70711   1.1441   1.8512        0  -0.80902   0.50000       0.0
x             0   1.8512   1.1441  0.70711  -0.80902  -0.50000       0.0
y        1.1441  0.70711        0   1.8512   0.80902  -0.50000       0.0
z        1.8512        0  0.70711   1.1441   0.80902   0.50000       0.0

In [18]:
df.loc['z']['XYZ'].Y.evalf(5).round(8)

0.50000

In [19]:
control = (fx.Z - fx.T).length()
half = fx.half
midface = (fx.Z + fx.Y)
gold    = half * fx.PHI * midface/midface.length()
Zi = gold + fx.J/fx.J.length() * (control/2)
Yi = gold + fx.M/fx.M.length() * (control/2)

In [20]:
control/2

1/2

In [21]:
gold + fx.J/fx.J.length() * (control/2)

ivm_vector(a=sqrt(2)/2 + sqrt(2)*(1/2 + sqrt(5)/2)/2, b=0, c=sqrt(2)/2, d=sqrt(2)*(1/2 + sqrt(5)/2)/2)

In [22]:
half

1/2

In [23]:
type(midface.norm0()[0])

sympy.core.numbers.One

In [24]:
gold

ivm_vector(a=sqrt(2)*(1/2 + sqrt(5)/2)/2, b=0, c=0, d=sqrt(2)*(1/2 + sqrt(5)/2)/2)

In [25]:
Zi

ivm_vector(a=sqrt(2)/2 + sqrt(2)*(1/2 + sqrt(5)/2)/2, b=0, c=sqrt(2)/2, d=sqrt(2)*(1/2 + sqrt(5)/2)/2)

In [26]:
Yi

ivm_vector(a=sqrt(2)*(1/2 + sqrt(5)/2)/2, b=sqrt(2)/2, c=0, d=sqrt(2)/2 + sqrt(2)*(1/2 + sqrt(5)/2)/2)

In [27]:
fx.M.length()

sqrt(2)/2

In [28]:
fx.A

ivm_vector(a=1, b=0, c=0, d=0)

In [29]:
type(fx.A.a)

sympy.core.numbers.One

In [69]:
tv.Svol.simplify() # S module volume

16/(1 + sqrt(5))**5

In [70]:
tv.Evol.simplify()  # E module volume

sqrt(2)/(1 + sqrt(5))**3

In [94]:
Sfactor = tv.Svol/tv.Evol
Sfactor.simplify()

-sqrt(10) + 3*sqrt(2)

In [95]:
cuboctavol, icosavol = fx.Cuboctahedron().volume, fx.Icosahedron().volume 

In [96]:
cuboctavol, icosavol.evalf(50)

(20, 18.512295868219161196009899292654531923571426913640)

In [97]:
cuboctavol/icosavol == Sfactor

True

In [98]:
IW = (5*half * Sfactor ** 2).simplify() # IW = 2.5 CO * sfactor * sfactor
IW

320/(1 + sqrt(5))**4

In [99]:
IW.evalf(50)

2.9179606750063091077247899380617129367814492116542

In [100]:
(IW + 24 * tv.Svol).simplify()

4